Reference: https://github.com/guoguo12/billboard-charts

In [25]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install billboard.py

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [13]:
from credentials import *

In [36]:
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
# Import Country Airplay chart from the week of September 28, 2019

chart = billboard.ChartData('country-airplay', date='2019-09-28')

In [6]:
chart.entries[0:5]

[billboard.ChartEntry(title='Living', artist='Dierks Bentley'),
 billboard.ChartEntry(title="I Don't Know About You", artist='Chris Lane'),
 billboard.ChartEntry(title="Knockin' Boots", artist='Luke Bryan'),
 billboard.ChartEntry(title='Southbound', artist='Carrie Underwood'),
 billboard.ChartEntry(title="The Ones That Didn't Make It Back Home", artist='Justin Moore')]

In [16]:
# Create new dataframe for Billboard Chart Data

chart_df = pd.DataFrame(columns=['song_title','artist','rank','date'])
max_allowed = 312
attempt = 0

while chart.previousDate:
    attempt +=1
    #Stops running after 5 years worth of data
    if attempt < max_allowed:
        for i in range(0,60):
            df = pd.DataFrame({'song_title':[chart[i].title], 'artist':[chart[i].artist], 'rank':[chart[i].rank], 'date':[chart.date]})
            chart_df = chart_df.append(df, ignore_index=True)
        #Sets chart date to previous week so it can pull in that week's data next
        chart=billboard.ChartData('country-airplay', chart.previousDate)
        #Sets a 5 second break between loops to prevent an error due to too many rapid requests
        time.sleep(5)
    else:
        break
    

In [17]:
chart_df.head()

,song_title,artist,rank,date
0,Living,Dierks Bentley,1,2019-09-28
1,I Don't Know About You,Chris Lane,2,2019-09-28
2,Knockin' Boots,Luke Bryan,3,2019-09-28
3,Southbound,Carrie Underwood,4,2019-09-28
4,The Ones That Didn't Make It Back Home,Justin Moore,5,2019-09-28


In [19]:
# Summary statistics
chart_df.describe()

,song_title,artist,rank,date
count,18660,18660,18660,18660
unique,869,295,60,311
top,Every Little Thing,Thomas Rhett,60,2017-12-09
freq,78,296,311,60


In [20]:
chart_df.head(10)

,song_title,artist,rank,date
0,Living,Dierks Bentley,1,2019-09-28
1,I Don't Know About You,Chris Lane,2,2019-09-28
2,Knockin' Boots,Luke Bryan,3,2019-09-28
3,Southbound,Carrie Underwood,4,2019-09-28
4,The Ones That Didn't Make It Back Home,Justin Moore,5,2019-09-28
5,Prayed For You,Matt Stell,6,2019-09-28
6,Love You Too Late,Cole Swindell,7,2019-09-28
7,Good Vibes,Chris Janson,8,2019-09-28
8,Beer Never Broke My Heart,Luke Combs,9,2019-09-28
9,What Happens In A Small Town,Brantley Gilbert + Lindsay Ell,10,2019-09-28


In [21]:
# Confirm dates starts at current week and goes back 5 years
chart_df.date.unique()

array(['2019-09-28', '2019-09-21', '2019-09-14', '2019-09-07',
       '2019-08-31', '2019-08-24', '2019-08-17', '2019-08-10',
       '2019-08-03', '2019-07-27', '2019-07-20', '2019-07-13',
       '2019-07-06', '2019-06-29', '2019-06-22', '2019-06-15',
       '2019-06-08', '2019-06-01', '2019-05-25', '2019-05-18',
       '2019-05-11', '2019-05-04', '2019-04-27', '2019-04-20',
       '2019-04-13', '2019-04-06', '2019-03-30', '2019-03-23',
       '2019-03-16', '2019-03-09', '2019-03-02', '2019-02-23',
       '2019-02-16', '2019-02-09', '2019-02-02', '2019-01-26',
       '2019-01-19', '2019-01-12', '2019-01-05', '2018-12-29',
       '2018-12-22', '2018-12-15', '2018-12-08', '2018-12-01',
       '2018-11-24', '2018-11-17', '2018-11-10', '2018-11-03',
       '2018-10-27', '2018-10-20', '2018-10-13', '2018-10-06',
       '2018-09-29', '2018-09-22', '2018-09-15', '2018-09-08',
       '2018-09-01', '2018-08-25', '2018-08-18', '2018-08-11',
       '2018-08-04', '2018-07-28', '2018-07-21', '2018-

In [22]:
# Drop duplicates, keeping only the highest ranking week for each song

chart_df = chart_df.sort_values(by='rank', ascending=True).drop_duplicates(subset='song_title')

In [23]:
# Reset index after dropping rows
chart_df.reset_index(drop=True, inplace=True)

In [24]:
# Summary statistics - confirm row count is lower, as expected
chart_df.describe()

,song_title,artist,rank,date
count,869,869,869,869
unique,869,294,60,293
top,Written In The Sand,Luke Bryan,1,2019-09-28
freq,1,17,207,39


In [26]:
# Clean artist names to allow songs to match with Spotify data

for i in range(len(chart_df)-1):
    if 'Featuring' in chart_df.artist[i]:
        chart_df.artist[i]=chart_df.artist[i].split("Featuring", maxsplit=1)[0]
    if 'With' in chart_df.artist[i]:
        chart_df.artist[i]=chart_df.artist[i].split("With", maxsplit=1)[0]
    if 'Duet' in chart_df.artist[i]:
        chart_df.artist[i]=chart_df.artist[i].split("Duet", maxsplit=1)[0]
    if ',' in chart_df.artist[i]:
        chart_df.artist[i]=chart_df.artist[i].split(",", maxsplit=1)[0]
    if '+' in chart_df.artist[i]:
        chart_df.artist[i]=chart_df.artist[i].split(",", maxsplit=1)[0]
    if  chart_df.artist[i]=='David Lee Murphy & Kenny Chesney':
        chart_df.artist[i]='David Lee Murphy'
    if  chart_df.artist[i]=='Tim McGraw & Faith Hill':
        chart_df.artist[i]='Tim McGraw'
    if  chart_df.artist[i]=='Jerrod Niemann & Lee Brice':
        chart_df.artist[i]='Jerrod Niemann'
    if  chart_df.artist[i]=='Reba McEntire & Lauren Daigle':
        chart_df.artist[i]='Reba McEntire'
    if  chart_df.artist[i]=='Bebe Rexha & Florida Georgia Line':
        chart_df.artist[i]='Bebe Rexha'
    if  chart_df.artist[i]=='Garth Brooks & Blake Shelton':
        chart_df.artist[i]='Garth Brooks'
    if  chart_df.artist[i]=='Willie Robertson & Luke Bryan':
        chart_df.artist[i]='Willie Robertson'

In [28]:
# Remove songs not on Spotify

chart_df = chart_df[chart_df.artist != 'Garth Brooks']
chart_df = chart_df[chart_df.artist != 'Tucker Beathard']
chart_df = chart_df[chart_df.artist != 'Candi Carpenter']
chart_df = chart_df[chart_df.song_title != 'A Few More Cowboys']
chart_df = chart_df[chart_df.song_title != 'PrizeFighter']
chart_df = chart_df[chart_df.song_title != 'Just Another Love Song']
chart_df = chart_df[chart_df.song_title != 'One Night Between Friends']
chart_df = chart_df[chart_df.song_title != 'Santa Baby']

In [30]:
# Reset index after dropping rows
chart_df.reset_index(drop=True, inplace=True)

In [31]:
chart_df.head()

,song_title,artist,rank,date
0,Living,Dierks Bentley,1,2019-09-28
1,"Sunrise, Sunburn, Sunset",Luke Bryan,1,2018-09-22
2,Burnin' It Down,Jason Aldean,1,2014-11-08
3,Leave The Night On,Sam Hunt,1,2014-11-15
4,Life Changes,Thomas Rhett,1,2018-09-08


In [32]:
chart_df

,song_title,artist,rank,date
0,Living,Dierks Bentley,1,2019-09-28
1,"Sunrise, Sunburn, Sunset",Luke Bryan,1,2018-09-22
2,Burnin' It Down,Jason Aldean,1,2014-11-08
3,Leave The Night On,Sam Hunt,1,2014-11-15
4,Life Changes,Thomas Rhett,1,2018-09-08
5,Neon Light,Blake Shelton,1,2014-11-22
6,Drowns The Whiskey,Jason Aldean,1,2018-09-01
7,Somewhere In My Car,Keith Urban,1,2014-12-06
8,Mercy,Brett Young,1,2018-08-18
9,Girl In A Country Song,Maddie & Tae,1,2014-12-20


In [41]:
# Create four blank lists
# Then append to them using the results from the Spotify search of the songs in the Billboard chart dataframe
# Create a none counter to identify every time the search fails to return any results

artist_name=[]
track_name=[]
track_id=[]
popularity=[]
None_counter=0
for i in range(len(chart_df)-1):
    track_results = sp.search(q=('artist:' + chart_df.artist[i] + ' track: ' + chart_df.song_title[i]), limit=1, type='track')
    for i, t in enumerate(track_results['tracks']['items']):
        if t == None:
            None_counter = None_counter + 1
        else:
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            popularity.append(t['popularity'])

In [46]:
# Verify all songs matched with Spotify results

None_counter

0

In [43]:
# Create a dataframe from the four lists

spotify_df = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(spotify_df.shape)
spotify_df.head()

(812, 4)


,artist_name,track_name,track_id,popularity
0,Dierks Bentley,Living,54K7AZoGpQklpygKrxZlKN,75
1,Luke Bryan,"Sunrise, Sunburn, Sunset",53yTYusPQJ1AApL1hi0Dnc,70
2,Jason Aldean,Burnin' It Down,5HJqpDspKDKwQpLjvkcIsD,65
3,Sam Hunt,Leave The Night On,3wx2kQWPn9p5UppQbNhPAk,66
4,Thomas Rhett,Life Changes,4Vxu50qVrQcycjRyJQaZLC,70


In [53]:
audio_features_df = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

for i in range(len(spotify_df)-1):
    audio_features_df = audio_features_df.append(sp.audio_features(tracks=[spotify_df.track_id[i]]))

In [54]:
audio_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.517,0.813,1,-6.325,1,0.0558,0.0153,0,0.1060,0.444,83.005,audio_features,54K7AZoGpQklpygKrxZlKN,spotify:track:54K7AZoGpQklpygKrxZlKN,https://api.spotify.com/v1/tracks/54K7AZoGpQkl...,https://api.spotify.com/v1/audio-analysis/54K7...,222147,4
0,0.472,0.884,9,-2.306,1,0.0584,0.1850,0,0.1780,0.718,159.960,audio_features,53yTYusPQJ1AApL1hi0Dnc,spotify:track:53yTYusPQJ1AApL1hi0Dnc,https://api.spotify.com/v1/tracks/53yTYusPQJ1A...,https://api.spotify.com/v1/audio-analysis/53yT...,216093,4
0,0.681,0.795,6,-6.652,1,0.0490,0.3050,9.41e-06,0.1090,0.640,135.939,audio_features,5HJqpDspKDKwQpLjvkcIsD,spotify:track:5HJqpDspKDKwQpLjvkcIsD,https://api.spotify.com/v1/tracks/5HJqpDspKDKw...,https://api.spotify.com/v1/audio-analysis/5HJq...,219160,4
0,0.514,0.951,9,-3.813,1,0.0603,0.0951,0,0.3490,0.838,171.960,audio_features,3wx2kQWPn9p5UppQbNhPAk,spotify:track:3wx2kQWPn9p5UppQbNhPAk,https://api.spotify.com/v1/tracks/3wx2kQWPn9p5...,https://api.spotify.com/v1/audio-analysis/3wx2...,192160,4
0,0.687,0.842,7,-4.382,1,0.0580,0.1050,0,0.0436,0.778,87.968,audio_features,4Vxu50qVrQcycjRyJQaZLC,spotify:track:4Vxu50qVrQcycjRyJQaZLC,https://api.spotify.com/v1/tracks/4Vxu50qVrQcy...,https://api.spotify.com/v1/audio-analysis/4Vxu...,190227,4


In [55]:
# Merge audio features dataframe with basic Spotify dataframe

track_df = audio_features_df.merge(spotify_df, left_on='id', right_on='track_id')

In [56]:
track_df.shape

(811, 22)

In [57]:
track_df.columns.tolist()

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'type',
 'id',
 'uri',
 'track_href',
 'analysis_url',
 'duration_ms',
 'time_signature',
 'artist_name',
 'track_name',
 'track_id',
 'popularity']

In [58]:
column_order = ['track_id',
'artist_name',
'track_name',
'rank',
'danceability',
'energy',
'key',
'loudness',
'mode',
'speechiness',
'acousticness',
'instrumentalness',
'liveness',
'valence',
'tempo',
'duration_ms',
'time_signature']

In [59]:
# Reorder columns

track_df = track_df.reindex(columns=column_order)

In [60]:
# Drop row with artist = The Karaoke Channel, as it is not correctly matched to the charting song

track_df = track_df[track_df['artist_name'] != 'The Karaoke Channel']

In [61]:
# Reset index after dropping row

track_df.reset_index(drop=True, inplace=True)

In [62]:
# Manually create a list of duos/groups

groups=['Maddie & Tae',
 'Old Dominion',
 'Florida Georgia Line',
 'Lady Antebellum',
 'Zac Brown Band',
 'Dan + Shay',
 'Eli Young Band',
 'Parmalee',
 'Rascal Flatts',
 'Little Big Town',
 'LANCO',
 'A Thousand Horses',
 'LOCASH',
 'Brothers Osborne',
 'Midland',
 'Thompson Square',
 'High Valley',
 'Big & Rich',
 'Runaway June',
 'The Band Perry',
 'The Swon Brothers',
 'Sugarland',
 'Gloriana',
 'Artists Of Then, Now & Forever',
 'The Cadillac Three',
 'Love and Theft',
 'The Henningsens',
 'Josh Abbott Band',
 'Montgomery Gentry',
 'Brooks & Dunn',
 'Gone West',
 'American Young',
 "Chasin' Crazy",
 'The Railers',
 'James Barker Band',
 'Radio Romance',
 'The Last Bandoleros',
 'King Calaway',
 'Waterloo Revival',
 'Pistol Annies',
 'Breaking Southwest',
 'Sir Rosevelt',
 'Everette',
 'Blackjack Billy',
 'Sister C',
 'Bobby Bones & The Raging Idiots']

In [66]:
# Manually create a list of artist that are female (solo or group) and mixed gender groups.

female=['Maddie & Tae', 
 'Bebe Rexha', 
 'Maren Morris', 
 'Lauren Alaina', 
 'Kelsea Ballerini', 
 'Carrie Underwood', 
 'Carly Pearce', 
 'Cam', 
 'Miranda Lambert', 
 'Jana Kramer', 
 'Taylor Swift', 
 'Cassadee Pope', 
 'Danielle Bradberry', 
 'RaeLynn',
 'Sara Evans',
 'Lindsay Ell',
 'Trisha Yearwood',
 'Jennifer Nettles',
 'Ingrid Andress',
 'Sheryl Crow',
 'Reba McEntire'
 'Ashley McBryde',
 'Kacey Musgraves',
 'Mickey Guyton',
 'Shania Twain',
 'Leah Turner',
 'Brandy Clark',
 'Caylee Hammack',
 'Natalie Stovall',
 'Clare Dunn',
 'Maggie Rose',
 'Martina McBride',
 'Lucy Hale',
 'Brooke Eden',
 'Kelleigh Bannen',
 'Kellie Pickler',
 'Kelly Clarkson',
 'Pistol Annies',
 'Lucy Angel',
 'P!nk',
 'Ashley Monroe',
 'Jillian Jacqueline',
 'Gabby Barrett',
 'Caroline Jones',
 'Rachel Wammack',
 'Jamie Lynn Spears',
 'Ashley Campbell',
 'Stephanie Quayle',
 'Shakira',
 'Kristy Lee Cook',
 'Abby Anderson',
 'LeAnn Rimes',
 'Bailey Bryan',
 'Sister C',
 'Tara Thompson']

mixed=['Lady Antebellum',
 'Little Big Town',
 'Thompson Square',
 'The Band Perry',
 'Sugarland',
 'Gloriana',
 'The Henningsens',
 'Gone West',
 'American Young',
 'The Railers',
 'Breaking Southwest',
 'Artists Of Then, Now & Forever']

In [63]:
# Create an is_group column using the list created above

track_df['is_group']=np.nan

for i in range(len(track_df)-1):
    if track_df['artist_name'][i] in groups:
        track_df['is_group'][i]=1
    else:
        track_df['is_group'][i]=0       

In [64]:
track_df['is_group'].value_counts()

0.0    651
1.0    159
Name: is_group, dtype: int64

In [67]:
# Create a gender column using the lists created above

track_df['gender']=np.nan

for i in range(len(track_df)-1):
    if track_df['artist_name'][i] in female:
        track_df['gender'][i]='female'
    elif track_df['artist_name'][i] in mixed:
        track_df['gender'][i]='mixed'
    else:
        track_df['gender'][i]='male'

In [68]:
track_df['gender'].value_counts()

male      647
female    127
mixed      36
Name: gender, dtype: int64

In [69]:
track_df.describe(include='all')

,track_id,artist_name,track_name,rank,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,is_group,gender
count,811,811,811,0.0,811.000000,811.000000,811.0,811.000000,811.0,811.000000,811.000000,811.0,811.000000,811.000000,811.000000,811.0,811.0,810.000000,810
unique,811,224,811,NaN,NaN,NaN,12.0,NaN,2.0,NaN,NaN,284.0,NaN,NaN,NaN,784.0,4.0,NaN,3
top,54RCDE1TAewECyX4WDUVm1,Jason Aldean,Homesick,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,190587.0,4.0,NaN,male
freq,1,17,1,NaN,NaN,NaN,99.0,NaN,718.0,NaN,NaN,502.0,NaN,NaN,NaN,3.0,773.0,NaN,647
mean,NaN,NaN,NaN,NaN,0.568998,0.736974,NaN,-5.442958,NaN,0.047944,0.169854,NaN,0.171041,0.557948,125.077390,NaN,NaN,0.196296,NaN
std,NaN,NaN,NaN,NaN,0.093859,0.156694,NaN,1.852261,NaN,0.060612,0.201136,NaN,0.114737,0.203722,31.543363,NaN,NaN,0.397441,NaN
min,NaN,NaN,NaN,NaN,0.169000,0.062800,NaN,-17.077000,NaN,0.023000,0.000226,NaN,0.021200,0.067300,48.718000,NaN,NaN,0.000000,NaN
25%,NaN,NaN,NaN,NaN,0.511000,0.657500,NaN,-6.289000,NaN,0.030600,0.027150,NaN,0.097200,0.406000,97.996500,NaN,NaN,0.000000,NaN
50%,NaN,NaN,NaN,NaN,0.573000,0.777000,NaN,-5.229000,NaN,0.036600,0.088300,NaN,0.122000,0.560000,122.981000,NaN,NaN,0.000000,NaN
75%,NaN,NaN,NaN,NaN,0.632000,0.856000,NaN,-4.251500,NaN,0.047800,0.252000,NaN,0.220000,0.717000,150.111500,NaN,NaN,0.000000,NaN


In [ ]:
# Merge dataframe created above with the Billboard Chart dataframe to create a single, comprehensive dataframe

## Working on better method to do this when creating the Spotify dataframe, as this loses too many records in the matching process

track_df = pd.merge(track_df, chart_df, left_on=['track_name','artist_name'], right_on = ['song_title','artist'])

In [74]:
track_df.to_csv('./track_df.csv')